In [99]:
from os import listdir
import pandas as pd
import numpy as np

from nltk.metrics.distance import edit_distance

In [100]:
MISSING = " "

# Load Data

In [101]:
files = listdir("../chart")

dfs_chart = {}
for f in files:
    year = f.split("_")[0]
    month = f.split("_")[1].split(".")[0]

    if dfs_chart.get(year) is None:
        dfs_chart[year] = {}
        
    dfs_chart[year][month] = pd.read_csv("../chart/" + f)

In [102]:
dfs_chart["2006"]["Janvier"]

,Rank,Artist,Music,Artist_wiki,Naissance,Pays d'origine,Origine,Nationalité,Pays
0,1,Juanes,La Camisa Negra,Juanes,"9 août 1972 (50 ans)Medellin, Colombie",NaN,NaN,NaN,NaN
1,2,Madonna,Hung Up,Madonna,"16 août 1958 (64 ans)Bay City, Michigan (États...",NaN,NaN,Américaine,NaN
2,3,Johnny Hallyday,Mon Plus Beau Noël,Johnny Hallyday,15 juin 1943 Paris 9e (France),NaN,NaN,Française,NaN
3,4,Tina Arena,Aimer Jusqu'à L'impossible,Tina Arena,"1er novembre 1967 (55 ans)Melbourne, Australie",NaN,NaN,NaN,NaN
4,5,Star Academy,Santiano,Star Academy (France),NaN,NaN,NaN,NaN,France
...,...,...,...,...,...,...,...,...,...
95,96,Jenifer,Serre-moi,Jenifer,15 novembre 1982 (40 ans)Nice,NaN,NaN,NaN,NaN
96,97,Lucie Bernardoni,"Petit Rat, Petit Loup",Lucie Bernardoni,"4 février 1987 (35 ans)Nice, Alpes-Maritimes, ...",NaN,NaN,NaN,NaN
97,98,Ilona Mitrecey,C'est Les Vacances,Ilona Mitrecey,1er septembre 1993 (29 ans)Fontenay-aux-Roses ...,NaN,NaN,NaN,NaN
98,99,Kanye West,Gold Digger,Kanye West,"8 juin 1977 (45 ans)Atlanta, Géorgie (États-Unis)",NaN,NaN,NaN,NaN


In [103]:
artist = pd.read_csv("../artist.csv", index_col=0)

In [104]:
artist

,Artist_wiki,Naissance,Pays d'origine,Origine,Nationalité,Pays
Artist,,,,,,
Anuel Aa,Anuel AA,26 novembre 1992 (30 ans)Carolina (Porto Rico),NaN,NaN,portoricaine,NaN
Childish Gambino,This Is America,NaN,NaN,NaN,NaN,NaN
Tydiaz,Foire de Châlons,NaN,NaN,NaN,NaN,NaN
Lucenzo,Lucenzo,"27 mai 1983 (39 ans)Bordeaux, France",NaN,NaN,NaN,NaN
Chris Garcia,Dany Garcia,29 novembre 1968 (54 ans)Belleville,NaN,NaN,américaine,NaN
...,...,...,...,...,...,...
Radio Killer,Radio Killer,NaN,NaN,NaN,NaN,NaN
Star Academy Maghreb,Star Academy (Maghreb),NaN,NaN,NaN,NaN,NaN
Mr. Oizo,Nonfilm,NaN,NaN,NaN,NaN,france


# Analysis

In [105]:
chart = []
for k1 in dfs_chart.keys():
    for k2 in dfs_chart[k1].keys():
        chart.append(dfs_chart[k1][k2])
chart = pd.concat(chart)
chart.drop(columns=["Rank"], inplace=True)
chart.drop_duplicates(inplace=True)

In [106]:
artist["distance"] = artist.apply(lambda x: edit_distance(x.name, str(x["Artist_wiki"])), axis=1)

In [107]:
n = 450
artist[artist.isna().sum(axis=1) >= 5].sort_values("distance")[n-50:n]

,Artist_wiki,Naissance,Pays d'origine,Origine,Nationalité,Pays,distance
Artist,,,,,,,
Passymal,Hardcore (chanson),NaN,NaN,NaN,NaN,NaN,16
Les Anges De La Téléréalité 5,Téléréalité,NaN,NaN,NaN,NaN,NaN,18
Roch Voisine Et CUr De Pirate,Boucle d'oreille,NaN,NaN,NaN,NaN,NaN,23
Philippe Tailleferd,Les Randonneurs à Saint-Tropez,NaN,NaN,NaN,NaN,NaN,26
Sidaction (Florent Pagny / Calogero / Pascal Obispo),Saison 3 de Star Academy (France),NaN,NaN,NaN,NaN,NaN,38


In [108]:
chart[chart["Artist"] == "Sarah"]

,Artist,Music,Artist_wiki,Naissance,Pays d'origine,Origine,Nationalité,Pays
58,Sarah,Le Meilleur Des Deux (the Best Of Both Worlds),Sarah,NaN,NaN,NaN,NaN,NaN


In [109]:
for col in ["Pays", "Origine", "Nationalité", "Pays d'origine"]:
    print("-" * 50)
    print(col.upper())
    print("-" * 50)
    print(artist[col].value_counts())

--------------------------------------------------
PAYS
--------------------------------------------------
france               181
états-unis           136
royaume-unis          72
allemagne             22
canada                12
pays-bas               9
australie              8
angleterre             6
suède                  6
italie                 5
corée                  5
espagne                4
belgique               4
inde                   4
irlande                3
finlande               3
autriche               3
moldavie               3
danemark               2
écosse                 2
ghana                  2
hongrie                2
biélorussie            2
burkina faso           2
portugal               2
norvège                2
roumanie               2
seychelles             1
afrique du sud         1
50                     1
côte d'ivoire          1
russie                 1
suisse                 1
détroit  michigan      1
jamaïque               1
mauritanie        